# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1014 09:16:56.744235    9689 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1014 09:17:04.068714    9717 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Fri Oct 14 09:17:06 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [26]:
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

# EXPERIMENT_FILE="../project/configs/project_arrival_config_comb_6.json"
# CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [27]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1014 09:34:47.345225   11118 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1014 09:34:56.498463   11143 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Fri Oct 14 09:35:00 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [28]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1014 09:35:15.373893   11172 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-14-2022 07:35:08 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-14-2022 07:35:08 root         INFO     Starting in cluster mode.
10-14-2022 07:35:08 root         INFO     Starting with experiment replication: 0 with seed: 42
10-14-2022 07:35:08 root         INFO     Starting as Orchestrator
10-14-2022 07:35:08 root         INFO     Starting Orchestrator, initializing resources....
10-14-2022 07:35:08 root         INFO     Loading in cluster configuration file
10-14-2022 07:35:08 root         INFO     Pointing configuration to in cluster configuration.
10-14-2022 07:35:08 root         INFO     Starting cluster manager
10-14-2022 07:35:08 ClusterManager INFO     Spinning up cluster manag

In [36]:
# To get logs from learners (example)
kubectl logs -n test trainjob-e866d960-d4a7-4aeb-870c-791fd03c6454-master-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-e866d960-d4a7-4aeb-870c-791fd03c6454-worker-0

W1014 09:39:30.163648   11683 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-14-2022 07:35:15 root         INFO     Loading file config/configuration.fltk.json
10-14-2022 07:35:15 root         INFO     Starting in client mode
10-14-2022 07:35:15 root         INFO     Starting with host=trainjob-e866d960-d4a7-4aeb-870c-791fd03c6454-master-0 and port=23456
10-14-2022 07:35:15 root         INFO     Initializing backend for training process: gloo
10-14-2022 07:35:18 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-14-2022 07:35:18 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-14-2022 07:35:18 root         INFO     Starting Creating client with 0
10-14-2022 07:35:18 Client-0-e

       0.99      , 0.5234192 , 0.9229249 , 0.96428571, 0.91743119]), class_recall=array([0.72427984, 0.96215139, 0.6498994 , 0.86401674, 0.69628099,
       0.92696629, 0.87647059, 0.93027888, 0.98580122, 0.97276265]), confusion_mat=array([[352,   1,   2,  11,   1,   1, 114,   0,   4,   0],
       [  0, 483,   0,  13,   2,   0,   4,   0,   0,   0],
       [  3,   0, 323,   4,  40,   0, 125,   0,   2,   0],
       [  5,   8,   2, 413,   7,   0,  42,   0,   1,   0],
       [  0,   1,  11,  12, 337,   0, 119,   0,   4,   0],
       [  0,   0,   0,   0,   0, 495,   0,  26,   0,  13],
       [ 25,   1,   6,  10,  16,   0, 447,   0,   5,   0],
       [  0,   0,   0,   0,   0,   2,   0, 467,   1,  32],
       [  0,   0,   1,   0,   1,   0,   3,   2, 486,   0],
       [  0,   0,   0,   0,   0,   2,   0,  11,   1, 500]]))]


## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [37]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1014 09:40:01.790837   11721 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1014 09:40:02.608283   11735 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [38]:
helm uninstall -n test experiment-orchestrator

W1014 09:45:32.256433   12262 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [39]:
helm uninstall extractor -n test

W1014 09:45:41.942740   12278 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [40]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1014 09:46:00.910739   12306 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-2cb508b7-8af0-47f7-a864-f59384c79ce0" deleted
pytorchjob.kubeflow.org "trainjob-e866d960-d4a7-4aeb-870c-791fd03c6454" deleted
pytorchjob.kubeflow.org "trainjob-edd34fe6-9999-4b33-ae31-48131ad97a47" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
